## Use psycopg2 engine to extract charts events for patient data

This script connects to the patient database 'extumate' and extracts chart events for the labelled patients identified by the field, hadm_id, in the table 'sample_vents'.

The script utilizes the pandas chunksize argument in order to avoid memory issues.

Finally, the data is stored with using `pd.DataFrame.to_feather` so it can be stored for future processing.

In [2]:
import sys
sys.path.append("../extumate")

from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import psutil
import os

from extumate.config import data_dir, extumate_engine_url

#### Set user-defined variables

In [3]:
data_dir

'../data/feathered/'

In [ ]:
export_name = "pulseox"
#feather_folder = "../data/feathered/"
export_path = data_dir+export_name
export_path


#### Write sql query

In [ ]:
sql_query = """
SELECT 
  chartevents.* 
FROM
  chartevents
  INNER JOIN sample_vents ON chartevents.hadm_id = sample_vents.hadm_id
WHERE
  (
    chartevents.itemid = 220277
  );
"""

#### print virtual memory available

In [ ]:
svmem = psutil.virtual_memory()
print (svmem.available) #in bytes 

#### print size of database we're pulling from

In [ ]:
os.path.getsize('../data/raw/chartevents.csv') 

#### figure out chunk size for pandas dataframe reading

In [ ]:
df_sample = pd.read_csv('../data/raw/chartevents.csv', nrows=10)
df_sample_size = df_sample.memory_usage(index=True).sum()
my_chunk = (2000000000 / df_sample_size)/10
my_chunk = int(my_chunk//1) # we get the integer part
print (my_chunk)

In [ ]:
# Define a database name (we're using a dataset on births, so we'll call it birth_db)

# Set your postgres username/password, and connection specifics

username = 'postgres'

password = 'password'    # change this

host     = 'localhost'

port     = '5432'            # default port that postgres listens on

db_name  = 'extumate'

#db_name  = 'birth_db'

In [ ]:
## 'engine' is a connection to a database
## Here, we're using postgres, but sqlalchemy can connect to other things too.
engine = create_engine( 'postgresql://{}:{}@{}:{}/{}'.format(username, password, host, port, db_name) )
print(engine.url)

#### Check engine is working by checking for 'sample_vents' table

In [ ]:
engine.has_table('sample_vents')

#### Connect using psycopg2 connection and query the database. 

Joining 'chartevents' with the 'sample_vents' table on the field hadm_id (so only pulling from patients who were ventilated), before selecting the type of event using the chartevents.itemid speeds up extraction of this data.

In [ ]:
# Connect to make queries using psycopg2
con = None
con = psycopg2.connect(database = db_name, user = username, host=host,password=password)

df_result = pd.read_sql_query(sql_query,con,chunksize=my_chunk)
df_result

In [ ]:
concat_df = pd.concat(
    [chunk
    for chunk in df_result])

In [ ]:
concat_df

In [ ]:
##concat_df.to_sql('pulseox', engine, if_exists='replace',chunksize=my_chunk) ### very, very slow!

#### Feather dataframe for future processing

In [ ]:
concat_df.reset_index(inplace=True)

In [ ]:
concat_df.to_feather(export_path)